In [9]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
import numpy as np
import requests
import datetime
from datetime import datetime, timedelta, date
from dateutil.parser import parse
import matplotlib.pyplot as plt

#Twitter API imports
import tweepy as tw

# NLP & Sentiment imports
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

from dotenv import load_dotenv
load_dotenv()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Kris/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [62]:
############################################################
"""
    Authenticates the Alpaca API and Twitter
    Returns a pass/fail statement
"""
############################################################  
    
# Setting twitter access and api keys
bearer_token = os.getenv("TWITTER_BEARER_TOKEN")
consumer_key= os.getenv("TWITTER_API_KEY")
consumer_secret= os.getenv("TWITTER_SECRET_KEY")
access_token= os.getenv("TWITTER_ACCESS_TOKEN")
access_token_secret= os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

# authentication for twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# test authentication
try:
    twitter_api.verify_credentials()
    auth = "Twitter Authentication Verified"
except:
    auth = "Error During Twitter Authentication"
    
print(auth)

Twitter Authentication Verified


In [70]:
############################################################
"""
    Twitter: Scrape Tweets and Analyze Sentiment
"""
############################################################

def twitter_sentiment(search_word, date_since, items):
       
    # Initialize the VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    # initializing the tweets dataframe
    df = []
    
    # adding retweet filter to search words
    search_word = search_word + " -filter:retweets"
    
    # Fetch top tweets/hastags for given ticker
    tweets = tw.Cursor(twitter_api.search,
              q=search_word,
              lang="en",
              since=date_since
                    ).items(items)
    
    for tweet in tweets:
    
        #storing tweet text
        tweet_fetched = tweet.text 

        # Get date of tweet
        tweet_date = pd.Timestamp(tweet.created_at, tz="America/New_York").isoformat()
        
        try:
            sentiment = analyzer.polarity_scores(tweet_fetched)
            compound = sentiment["compound"]
            #pos = sentiment["pos"]
            #neu = sentiment["neu"]
            #neg = sentiment["neg"]
        
            df.append({
                "date": tweet_date,
                "tweet": tweet_fetched,
                "compound": compound,
                #"positive": pos,
                #"negative": neg,
                #"neutral": neu
            
            })
        
        except AttributeError:
            pass
    
    df = pd.DataFrame(df)
    
    return df

############################################################
"""
    get_twitter_sentiment(search_words):
    Takes 2 argument, word(s) to search tweets for, and number of items (tweets) to return.
    
    Scrapes Twitter for given search words in tweet
    Calculates compound sentiment with VADER sentiment analyzer on each tweet
    Calculates average compound sentiment score each 1 hour
    Normalizes average hourly VADER compound score
    Returns Average Hourly Sentiment Dataframe with Columns: 
"""
############################################################

def get_avg_twitter_sentiment(ticker, search_word):

    # tweepy variables
    search_word = search_word
    date_since = "2021-01-01"
    items = 1000
    twitter_search_phrase = search_word + " " + ticker

    # call the twitter sentiment function and return a dataframe
    tweets_df = twitter_sentiment(twitter_search_phrase, date_since, items)

    # Changes the date column to proper datetime format
    tweets_df['date'] = pd.to_datetime(tweets_df['date'])
    
    # Grouping the tweets by Hour and taking their average Hourly sentiment
    avg_hourly_sentiment = tweets_df.groupby(pd.Grouper(key='date', freq='H')).mean().dropna()
    
    # Get the normalized sentiment score of -1, 0, 1
    # avg_hourly_sentiment["normalized"] = avg_hourly_sentiment["compound"].apply(lambda x : get_normalized(x))
    
    return avg_hourly_sentiment


############################################################
"""
    Sentiment calculation based on compound score
"""
############################################################

def get_normalized(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.04:  # Positive
        result = 1
    elif score <= -0.04:  # Negative
        result = -1

    return result


############################################################
    """
    This function accepts the column number for the features (X) .
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns a numpy array of X.
    
    `df`: The original DataFrame with the time series data.
    `window`: The window size in days of previous closing prices that will be used for the prediction.
    `feature_col_number`: The column number from the original DataFrame where the features are located.
  
    """
############################################################

def window_data(df, window, feature_col_number):

    X = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        X.append(features)
    return np.array(X)


############################################################
"""
    Machine Learning Model
    Input the Sentiment data and return predicted stock returns
"""
############################################################

def get_predictions(combined_df):
    
    window_size = 1
    X = window_data(combined_df, window_size, 0)

    # Use the MinMaxScaler to scale data between 0 and 1.
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)

    # Reshape the features for the model
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Load the sentiment model 
    from tensorflow.keras.models import model_from_json

    # load json and create model
    file_path = Path("ml_model/sentiment_model.json")
    with open(file_path, "r") as json_file:
        sentiment_model_json = json_file.read()
    sentiment_loaded_model = model_from_json(sentiment_model_json)
    
    # load weights into new model
    file_path = "ml_model/sentiment_model.h5"
    sentiment_loaded_model.load_weights(file_path)
    
    # Make some predictions with the loaded model
    predicted = sentiment_loaded_model.predict(X)
    
    # Recover the original prices instead of the scaled version
    predicted_prices = scaler.inverse_transform(predicted)
    
    # Create a DataFrame of Predicted values
    predicted_returns = pd.DataFrame({
        "Predicted": predicted_prices.ravel()
        })

    return predicted_returns


############################################################
"""
    main_function:
    Defines the main function.
    1 argument: search word to search twitter and stock prices.
"""
############################################################

def tweet_sentiment_main_function(ticker, search_word):
      
    twitter_sentiment = get_avg_twitter_sentiment(ticker, search_word)
    predicted_df_all = get_predictions(twitter_sentiment)
    predicted_df = predicted_df_all.head(5)
    
    return predicted_df

In [69]:
#search_word = "microsoft" #search_word
#ticker = "MSFT" #ticker

#predicted_df = tweet_sentiment_main_function(ticker, search_word)
#predicted_df

,Predicted
0,0.226018
1,0.226006
2,0.225919
3,0.226003
4,0.226089
